In [15]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_twoHeaded import Trainer_twoHeaded
from src.trainer.trainer_VAE import Trainer_VAE
from src.models.classifiers import *
from src.trainer.model_class import Model_class
from src.trainer.loss_class import *

from src.models.autoencoders import *
from src.models.joinedModel import *
from src.models.twoHeaded import *

from src.tools import upsampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
BATCH_SIZE = 1024
INIT_PARAM = 256
NUM_FEATURES = 15

In [17]:
df = pd.read_csv('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_importance/feature_imporstance_WS.csv')

In [18]:
features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df.to_csv('../data/df_to_enc.csv', index = False)

In [19]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42,
                                                 train_size=0.7)

In [20]:
X_train, y_train = upsampling(0.4,X_train, y_train) # 0.3

In [21]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [22]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [23]:
encoder = nn.Sequential(
            nn.Linear(NUM_FEATURES, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 15)
        )

In [24]:
Baseline_classifier(NUM_FEATURES, INIT_PARAM).classifier[0]

Linear(in_features=15, out_features=256, bias=True)

In [25]:
model = TwoHeadedModel(encoder,
                       Baseline_classifier(NUM_FEATURES, INIT_PARAM).classifier)

In [26]:
loss = Loss_class_TH(TwoHeadedLoss(2,FocalLoss(gamma=2)))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.90)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'fe_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}_BASELINE',
    project="Internship_project",
    dir = '../logs/'
)

In [27]:
trainer = Trainer_twoHeaded(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/classifier/',
                  saving_model=False
                  )

In [ ]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 11.15it/s]


Epoch: 1 of 40, 0.029 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  9.12it/s]


Epoch: 2 of 40, 0.029 min


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 12.57it/s]


Epoch: 39 of 40, 0.028 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 11.54it/s]


Epoch: 40 of 40, 0.033 min


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,█▇▇▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,█▂▁███▇▄▆▇▇▇▆▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇
test_auc_score,▇█▇▁▂▆▇█▇█▇█████████▇███████████████████
test_f1_score,▇▂▁▅▅█▇▄▆▇▇▇▆▇▆▇▇██▇█▇▆▆▇█▇▇▇▇▇▇▇█▇█▇▇▇▇
test_fpr,▂▇█▁▁▁▂▅▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
test_loss,▅▄█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,▄██▁▁▅▇█▇▇▇▇█▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train_accuracy,▁▅▆▆▆▆▆▆▇▇▇▇▇▇█▇██▇█▇▇██████████████████
train_auc_score,▁▄▄▄▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇████████████
train_f1_score,▁▅▇▇▇▇▇▇▇▇▇█████████████████████████████
